In [1]:
import pandas as pd
import os
import time

In [2]:
MAIN_DIR = "/Volumes/Extreme SSD"
DATA_DIR = os.path.join(MAIN_DIR, "data")

## Feedback information
#### Dropping duplicate rows

In [3]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'feedbacks.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'feedbacks.pickle')

df = pd.read_pickle(data_file)
mem = df.memory_usage(deep = True).sum()
print(len(df), len(df.drop_duplicates()), len(df) - len(df.drop_duplicates()))
df = df.drop_duplicates()

2275492 2152745 122747


In [4]:
def get_day(day):
    day = day.str.split(' ').str[0]
    day = day.replace('today', '0')
    pd.to_numeric(day)
    return day

def to_dt(col):
    return pd.to_datetime(col, unit = 's', errors = 'coerce')

In [5]:
df['freshness'] = pd.to_numeric(get_day(df.freshness))
df = df\
    .assign(rating = pd.to_numeric(df.rating.str.get(0)),
            rtime  = df.stime - df.freshness * 86400)\
    .assign(rtime_dt = lambda df_copy: to_dt(df_copy.rtime))

In [6]:
print('memory usage before cleaning                    ', mem)
print('memory usage before memory usage optimalization ', df.memory_usage(deep = True).sum())

column_types = {
    'name'     : 'category',
    'stime'    : 'float32',
    'rating'   : pd.Int16Dtype(),
    'feedback' : 'object',
    'freshness': 'float32',
    'rtime'    : 'float32',
    'rtime_dt' : 'datetime64'
}

df = df.astype(column_types)

print('memory usage after memory usage optimalization  ', df.memory_usage(deep = True).sum())

memory usage before cleaning                     890426282
memory usage before memory usage optimalization  639579799
memory usage after memory usage optimalization   460772248


#### dropping duplicates

In scaped data, the same data is often scraped multiple times. To assure that feedbacks are not included twice duplicates will be dropped. Feedbacks can be uniquely identified by: 
- `name`
- `rating`
- `feedback`
- `rtime_dt`

In [7]:
# drop duplicate feedbacks from table 
subset = ['name', 'rating', 'feedback', 'rtime_dt']
print("before dropping duplicates: {0}\nafter dropping duplicates:  {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))

df = df.drop_duplicates(subset = subset)
df = df.reset_index()

before dropping duplicates: 2152745
after dropping duplicates:  2145909

6836 duplicate cases were dropped


In [8]:
df.to_pickle(out_file)

## Vendor information


In [9]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'vendors.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'vendors.pickle')

df = pd.read_pickle(data_file)
mem = df.memory_usage(deep = True).sum()

In [10]:
def date_to_num(col):
    """
    This function returns a numeric value for valid date strings
    If the date is today, a zero-value is returned
    Otherwise a Null value is returned 
    """
    if isinstance(col, str):
        if 'today' not in col:
            trans_dict = {
                'year'  : 365.25,
                'month' : 365.25/12,
                'day'   : 1,
                'hour'  : 1/24
            }

            value = float(col.split(' ')[0])
            unit = col.split(' ')[1]
            for key in trans_dict.keys():
                if unit in key:
                    unit = unit.replace(key, str(trans_dict[key]))
            unit = float(unit)

            return value * unit
        
        elif 'today' in col:
            return 0
        
        else:
            return None

In [11]:
df = df\
    .assign(
        ctime     = df.ctime.str.replace("about ", "").str.replace('s', '').str.replace(' ago', ''),
        otime     = df.otime.str.replace("about ", "").str.replace('s', '').str.replace(' ago', ''))\
    .assign(
        ctime_num = lambda df_copy: df_copy['ctime'].apply(date_to_num),
        otime_num = lambda df_copy: df_copy['otime'].apply(date_to_num))\
    .assign(
        otime_dt  = lambda df_copy: to_dt(df_copy.stime - df_copy.otime_num * 86400),
        ctime_dt  = lambda df_copy: to_dt(df_copy.stime - df_copy.ctime_num * 86400),
        stime_dt  = to_dt(df.stime),
        score     = df.score.replace('NEW VENDOR', 0).apply(pd.to_numeric))\
    .assign(
        score     = lambda df_copy: df_copy.score.mask(df_copy.score > 100)
    )

In [12]:
# drop duplicate feedbacks from table 
df['flag'] = df.ctime_dt.dt.strftime('%d%m%Y').astype('category')
subset = ['name', 'flag', 'location', 'area']
print("before dropping duplicates: {0}\nafter dropping duplicates:   {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))
df = df.drop_duplicates(subset = subset)

before dropping duplicates: 12242
after dropping duplicates:  9377

2865 duplicate cases were dropped


In [13]:
columns = ['name', 'stime', 'stime_dt', 'score', 'ctime', 'ctime_num', 'ctime_dt', 
           'otime', 'otime_num', 'otime_dt', 'location', 'area']

df = df.reindex(columns, axis=1)
df = df.reset_index()

In [14]:
print('memory usage before cleaning                    ', mem)
print('memory usage before memory usage optimalization ', df.memory_usage(deep = True).sum())

column_types = {
    'name'     : 'category',
    'stime'    : 'float32',
    'stime_dt' : 'datetime64',
    'score'    : pd.Int16Dtype(),
    'ctime'    : 'category', 
    'ctime_num': 'float32', 
    'ctime_dt' : 'datetime64', 
    'otime'    : 'category', 
    'otime_num': 'float32', 
    'otime_dt' : 'datetime64', 
    'location' : 'category', 
    'area'     : 'category'
}

df = df.astype(column_types)

print('memory usage after memory usage optimalization   ', df.memory_usage(deep = True).sum())

memory usage before cleaning                     5708163
memory usage before memory usage optimalization  3695920
memory usage after memory usage optimalization    640278


In [15]:
df.to_pickle(out_file)

## Item Information

In [7]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'items.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'items.pickle')

df = pd.read_pickle(data_file)
print(len(df), len(df.drop_duplicates()), len(df) - len(df.drop_duplicates()))
df = df.drop_duplicates()

22003618 22003618 0


In [8]:
def clean_freshness(col):
    #assert that freshness is not missing
    new_col = []
    for i in col:
        if not isinstance(i, (float, int)) and 'day' in i and len(i) < 10:
            new_col.append(i)
        else:
            new_col.append(None)
    
    return pd.Series(new_col)

# mem = df.memory_usage(deep = True).sum()

In [11]:
df = df\
    .assign(
        vendor = df.vendor.str.replace('\\n', ''),
        freshness = clean_freshness(df.freshness).\
            str.replace('s', '')\
            .apply(date_to_num),
        rating = df.rating.str.get(0),
        price = df.price.str[1:],
        stime_str = df.stime_dt)\
    .assign(
        rtime = lambda df_copy: df_copy.stime - df_copy.freshness *86400)\
    .assign(
        rtime_dt = lambda df_copy: to_dt(df_copy.rtime),
        stime_dt = lambda df_copy: to_dt(df_copy.stime))

In [12]:
# print('memory usage before cleaning', mem)
# print('memory usage before memory usage optimalization', df.memory_usage(deep = True).sum())

column_types = {
    'vendor'   : 'category',
    'stime'    : 'float32',
    'stime_dt' : 'datetime64',
    'stime_str': 'category',
    'rating'   : 'float16',
    'feedback' : 'object',
    'item'     : 'object',
    'price'    : 'float32',
    'freshness': 'float32',
    'rtime'    : 'float32',
    'rtime_dt' : 'datetime64',
    'loc'      : 'category',
    'area'     : 'category'
}

df = df.astype(column_types)

# downcast integer variables
df = df\
    .assign(
        freshness = pd.Series(df.freshness, dtype=pd.Int16Dtype()),
        rating = pd.Series(df.rating, dtype=pd.Int16Dtype()))

print('memory usage after memory usage optimalization', df.memory_usage(deep = True).sum())

memory usage after memory usage optimalization 5984469841


In [13]:
# drop duplicate feedbacks from table 
df['flag'] = df.rtime_dt.dt.strftime('%d%m%Y').astype('category')
subset = ['vendor', 'flag', 'item', 'loc', 'feedback', 'rating']
print("before dropping duplicates: {0}\nafter dropping duplicates:  {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))
df = df.drop_duplicates(subset = subset)
df['location'] = df['loc']

before dropping duplicates: 22003618
after dropping duplicates:  1362588

20641030 duplicate cases were dropped


In [14]:
columns = ['vendor', 'stime', 'stime_dt', 'stime_str', 'rating', 'feedback', 'item', 'price',
         'freshness', 'rtime', 'rtime_dt', 'location', 'area']

df = df.reindex(columns, axis=1)
df = df.reset_index(drop = True)

In [15]:
df.sample(5)

,vendor,stime,stime_dt,stime_str,rating,feedback,item,price,freshness,rtime,rtime_dt,location,area
344253,sbraser,1.402431e+09,2014-06-10 20:08:45,2014-05-19,5,FE will update once I get it,0.1g MDMA TESTER - UK IMPORTED TAN CRYSTALS - ...,0.000000,146,1.389817e+09,2014-01-15 20:08:45,Australia,Australia
1261692,WeedGirlz93,1.414796e+09,2014-10-31 22:52:17,2014-11-01,5,Great Vendor. Great Quality. Would recommend.,1 Large Brownie,0.055221,43,1.411081e+09,2014-09-18 22:52:17,United States,United States
1135534,Swazibudbud88872,1.413275e+09,2014-10-14 08:24:20,2014-10-15,5,"nicely done, product matches description, no i...",5g Swazi Gold Kief,0.103207,57,1.408350e+09,2014-08-18 08:24:20,South Africa,Worldwide
1089262,bodyinaction86,1.412813e+09,2014-10-08 23:59:35,2014-10-11,5,Got to USA in exactly 2 weeks after being ship...,Arimidex Anastrozole Astrazeneca 28tabs 1mg/tab,0.735294,42,1.409184e+09,2014-08-27 23:59:35,Undeclared,Worldwide
164137,FelixUK,1.402430e+09,2014-06-10 19:58:51,2014-03-10,5,Took 11 days but I got IT!!!! Good product!,FelixUK (0.25g) - 99.8% Pure 2CB UK Domestic,0.051190,4,1.402085e+09,2014-06-06 19:58:51,United Kingdom,Worldwide


In [16]:
df.to_pickle(out_file)

# Categories

In [55]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'categories.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'categories.pickle')

df = pd.read_pickle(data_file)
mem = df.memory_usage(deep = True).sum()
print(len(df), len(df.drop_duplicates()), len(df) - len(df.drop_duplicates()))
df = df.drop_duplicates()

347877 325040 22837


In [57]:
# clean variables
df = df\
    .assign(title = df.title.str.strip(),
            vendor = df.vendor.str.strip(),
            location = df.location.str.strip(),
            area = df.location.str.strip(),
            price = pd.to_numeric(
                    df.price.str.strip()\
                    .str[1:],
                errors = 'coerce'),
            category = df.category.str.strip())

In [59]:
# memory optimization
column_types = {
    'title'    : 'object',
    'vendor'   : 'category',
    'location' : 'category',
    'area'     : 'category',
    'price'    : 'float64',
    'category' : 'category'
}

df = df.astype(column_types)

In [78]:
df.sample(5)

,title,vendor,location,area,price,category
106315,Dexedrine SR (Dextroamphetamine) 10mg Spansule...,Quixote,United States,United States,0.172319,drugs-prescription-stimulants-amphetamine-dexe...
97107,28g AK48 Indoor Organic Top Shelf Weed Dank!,KushDepot,United States,United States,0.607910,drugs-cannabis
305695,100g pure FU-144 with FREE worldwide shipping,drzheng,China,China,2.462049,drugs-cannabis-synthetic
36184,1 Milion Youtube High Quality Real Views!,profesorhouse,Belgium,Belgium,1.105088,money
162306,MDMA 7 GRAMS - PURE & UNCUT ROCKS,jerseycow,United Kingdom,United Kingdom,0.700870,drugs-ecstasy


In [77]:
# encode infinite values to missing and restructure. 
df['price']= df['price'].replace([np.inf, -np.inf], np.nan)
df = df.reset_index(drop = True)

# safe dataframe
df.to_pickle(out_file)